## Construction Heuristic

### Algorithm

1. sort parties in order of descending size
1. add parties to the timeslot k until the restaurant capacity is full or no more tables can be added

#### Pre-work: Randomly generating a unique HEX string of 6 digits to use as a file identifier and relate logging files to data files

In [1]:
import random
import string

RG_HEX_ID = ''.join(random.choice(string.hexdigits) for _ in range(6))
print(RG_HEX_ID)

FE1F4D


### Step 1: Parse input parameters

In [2]:
import json
import random

w = l = num_timeslots = randomly_generated_data = 0
marginal_revenue = {}

# set of timeslots - static
K = 7

with open('../data.json') as f:
    data = json.load(f)
    w = data['restaurant_width']
    l = data['restaurant_length']
    randomly_generated_data = data['randomly_generated_data']
    max_party_size = data['max_party_size']
    for i in data['marginal_revenue']:
        marginal_revenue[int(i)] = data['marginal_revenue'][i]
        
# Restaurant squared area
MAX_SIZE = l * w

### Step 2: Randomly generate data

In [20]:
# set of parties at each timeslot
# since there are 7 timeslots and since this data is randomly generated, 
# the set of parties during timelsot k will be I[k].
# Prior to having randomly generated demand we had set I = 40 (40 parties) for each timeslot
if randomly_generated_data == True:
    I = [
        random.randint(8,10), 
        random.randint(10,12),
        random.randint(12,14),
        random.randint(14,16),
        random.randint(14,16),
        random.randint(14,15),
        random.randint(10,12),
    ]

    print(I)
    
else:
    print("Please allow data to be randomly generated to ensure accuracy and reliability of results")


#initializing the party 
party_size = {}
for k in range(K):
    for i in range(I[k]):
        party_size[i, k] = random.randint(2, max_party_size)
        
print(f"{max_party_size}\n")
print(f"{party_size}")

[10, 11, 13, 15, 16, 15, 10]
8

{(0, 0): 2, (1, 0): 6, (2, 0): 5, (3, 0): 8, (4, 0): 8, (5, 0): 2, (6, 0): 2, (7, 0): 4, (8, 0): 7, (9, 0): 7, (0, 1): 8, (1, 1): 8, (2, 1): 2, (3, 1): 7, (4, 1): 6, (5, 1): 8, (6, 1): 5, (7, 1): 5, (8, 1): 5, (9, 1): 2, (10, 1): 4, (0, 2): 7, (1, 2): 5, (2, 2): 6, (3, 2): 7, (4, 2): 2, (5, 2): 8, (6, 2): 2, (7, 2): 7, (8, 2): 7, (9, 2): 6, (10, 2): 3, (11, 2): 2, (12, 2): 2, (0, 3): 8, (1, 3): 3, (2, 3): 7, (3, 3): 5, (4, 3): 5, (5, 3): 4, (6, 3): 6, (7, 3): 3, (8, 3): 5, (9, 3): 5, (10, 3): 8, (11, 3): 5, (12, 3): 7, (13, 3): 2, (14, 3): 6, (0, 4): 5, (1, 4): 3, (2, 4): 5, (3, 4): 7, (4, 4): 4, (5, 4): 5, (6, 4): 5, (7, 4): 8, (8, 4): 2, (9, 4): 2, (10, 4): 2, (11, 4): 8, (12, 4): 6, (13, 4): 8, (14, 4): 3, (15, 4): 2, (0, 5): 2, (1, 5): 2, (2, 5): 8, (3, 5): 5, (4, 5): 7, (5, 5): 7, (6, 5): 5, (7, 5): 8, (8, 5): 7, (9, 5): 8, (10, 5): 7, (11, 5): 7, (12, 5): 2, (13, 5): 5, (14, 5): 5, (0, 6): 2, (1, 6): 7, (2, 6): 8, (3, 6): 3, (4, 6): 6, (5, 6): 3, (

### Step 3: Calculate table sizes and space that each party will occupy if seated

In [4]:
# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = {}
for i in range(2,max_party_size + 1):
    table_mapping[i] = i // 2 + i % 2

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for k in range(K):
    for i in range(I[k]):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

### Step 4: Create and pre-populate data structures

In [5]:
sorted_space = space.copy()

sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[1], reverse=True)}
sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[0][1], reverse=False)}
print(f"{sorted_space}")

{(2, 0): 18, (4, 0): 18, (5, 0): 18, (0, 0): 15, (1, 0): 15, (6, 0): 15, (3, 0): 12, (8, 0): 12, (9, 0): 12, (7, 0): 9, (1, 1): 18, (2, 1): 18, (5, 1): 18, (9, 1): 18, (11, 1): 18, (0, 1): 15, (3, 1): 15, (8, 1): 15, (10, 1): 15, (4, 1): 12, (6, 1): 12, (7, 1): 9, (10, 2): 18, (11, 2): 18, (0, 2): 15, (1, 2): 15, (3, 2): 15, (4, 2): 15, (7, 2): 15, (9, 2): 15, (12, 2): 15, (2, 2): 12, (5, 2): 12, (6, 2): 12, (13, 2): 12, (8, 2): 9, (1, 3): 18, (4, 3): 18, (5, 3): 18, (11, 3): 18, (12, 3): 18, (0, 3): 15, (13, 3): 15, (2, 3): 12, (7, 3): 12, (8, 3): 12, (3, 3): 9, (6, 3): 9, (9, 3): 9, (10, 3): 9, (0, 4): 18, (4, 4): 18, (5, 4): 18, (8, 4): 18, (9, 4): 18, (11, 4): 18, (13, 4): 18, (2, 4): 15, (3, 4): 15, (6, 4): 15, (1, 4): 12, (7, 4): 12, (10, 4): 12, (12, 4): 12, (14, 4): 12, (1, 5): 18, (10, 5): 18, (11, 5): 18, (3, 5): 15, (7, 5): 15, (2, 5): 12, (6, 5): 12, (8, 5): 12, (14, 5): 12, (0, 5): 9, (4, 5): 9, (5, 5): 9, (9, 5): 9, (12, 5): 9, (13, 5): 9, (3, 6): 18, (6, 6): 18, (7, 6): 

In [6]:
# initializing a list of keys for the space dictionary
keys_list = list(sorted_space)


# looping over each of the time slot's sorted party sizes 
timeslot_sorted_parties = {}
global_counter = 0
for k in range(K):
    counter = 0
    sorted_parties=[]
    for key in sorted_space.keys():
        sorted_parties.append(sorted_space[keys_list[global_counter]])
        counter += 1
        global_counter += 1
        if(counter == I[k]):
            break
    timeslot_sorted_parties[k] = sorted_parties
print(f"{timeslot_sorted_parties}")
#print(f"{sorted_space}")
#print(f"{sorted_space[sorted_space[i],i]}")

x = [[0 for i in range(I[k])] for k in range(K)]

{0: [18, 18, 18, 15, 15, 15, 12, 12, 12, 9], 1: [18, 18, 18, 18, 18, 15, 15, 15, 15, 12, 12, 9], 2: [18, 18, 15, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 9], 3: [18, 18, 18, 18, 18, 15, 15, 12, 12, 12, 9, 9, 9, 9], 4: [18, 18, 18, 18, 18, 18, 18, 15, 15, 15, 12, 12, 12, 12, 12], 5: [18, 18, 18, 15, 15, 12, 12, 12, 12, 9, 9, 9, 9, 9, 9], 6: [18, 18, 18, 18, 15, 15, 15, 12, 12, 12, 12, 9]}


### Step 5: Construction Heuristic

In [7]:
import pprint
import time

start_time = time.time()

# Observation: Sometimes we have a party size of 3/5 instead of 4/6 get allocated even though 4 will produce more revenue.
# This is because the space occupied by each is the same.
# Way we can do this is by sorting at party_size instead of space and that will propogate.
binaries = []
incumbent = []

daily_revenue = 0
counter = 0
space_used = []
for k in range(K):
    state = []
    revenue = 0
    occupied_space = 0
    print(f"timeslot{k}")
    for i in range(I[k]):
        
        dict_to_append = {}
        size_of_party = party_size[keys_list[counter]]
        dict_to_append['party'] = i
        dict_to_append['timeslot'] = k
        dict_to_append['party_size'] = size_of_party
        
        if (occupied_space + timeslot_sorted_parties[k][i] < MAX_SIZE):
            occupied_space += timeslot_sorted_parties[k][i]
            x[k][i] = 1
            added_revenue = marginal_revenue[size_of_party] * size_of_party
            revenue += added_revenue
            #dict_to_append[i,k] = [1, added_revenue]
            binaries.append([i,size_of_party,k,1,added_revenue,revenue])
            dict_to_append['seated'] = 1
            #print(f"party size = {size_of_party}, marginal revenue = {marginal_revenue[size_of_party]}, rev={revenue}")
        else:
            dict_to_append['seated'] = 0
            binaries.append([i,size_of_party,k,'',0,revenue])
        counter+=1
        state.append(dict_to_append)
    incumbent.append(state)
    space_used.append(occupied_space)
#     print(f"{occupied_space}\n")
    daily_revenue += revenue
incumbent.append(state)
incumbent.append(daily_revenue)
incumbent.append(space_used)
# print(f"{incumbent}")

print("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))

timeslot0
timeslot1
timeslot2
timeslot3
timeslot4
timeslot5
timeslot6


--- 0.0009226799011230469 seconds ---




### Step 6: Export results to CSV

In [8]:
import csv
import string

file_name = test = "construction_heuristic_" + RG_HEX_ID + ".csv"
with open(file_name, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["party i", "size of Party i", "timeslot k", "seated (binary)", "added revenue", "cum. revenue"])
    for i in binaries:
       writer.writerow(i)

In [9]:
print(f"Total Revenue: ${incumbent[8]}")
print(f"Average Space Used per Timeslot: {sum(incumbent[9]) / 7} m^2")

total_guests = 0
seated_guests = 0
total_groups = 0
seated_groups = 0
party_size_histogram = {
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0
}
for i in range(K):
    for j, val in enumerate(incumbent[i]):
        party_size = val['party_size']
        party_size_histogram[party_size] += 1
        total_guests += party_size
        total_groups += 1
        if (val['seated'] == 1):
            seated_guests += party_size
            seated_groups += 1
for key, value in party_size_histogram.items():
    print(f"Party Size: {key}, occurrences: {value}")
    
print(f"Average table revenue: ${incumbent[8] / seated_groups}")
print(f"Percentage of groups seated: {seated_groups / total_groups * 100}%")

Total Revenue: $7620
Average Space Used per Timeslot: 96.85714285714286 m^2
Party Size: 2, occurrences: 14
Party Size: 3, occurrences: 14
Party Size: 4, occurrences: 11
Party Size: 5, occurrences: 11
Party Size: 6, occurrences: 13
Party Size: 7, occurrences: 15
Party Size: 8, occurrences: 14
Average table revenue: $185.85365853658536
Percentage of groups seated: 44.565217391304344%


#### Visualizing incumbent data structure

In [10]:
pprint.pprint(f"{incumbent}")

("[[{'party': 0, 'timeslot': 0, 'party_size': 7, 'seated': 1}, {'party': 1, "
 "'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 2, 'timeslot': 0, "
 "'party_size': 8, 'seated': 1}, {'party': 3, 'timeslot': 0, 'party_size': 5, "
 "'seated': 1}, {'party': 4, 'timeslot': 0, 'party_size': 6, 'seated': 1}, "
 "{'party': 5, 'timeslot': 0, 'party_size': 5, 'seated': 1}, {'party': 6, "
 "'timeslot': 0, 'party_size': 4, 'seated': 0}, {'party': 7, 'timeslot': 0, "
 "'party_size': 4, 'seated': 0}, {'party': 8, 'timeslot': 0, 'party_size': 4, "
 "'seated': 0}, {'party': 9, 'timeslot': 0, 'party_size': 2, 'seated': 0}], "
 "[{'party': 0, 'timeslot': 1, 'party_size': 8, 'seated': 1}, {'party': 1, "
 "'timeslot': 1, 'party_size': 7, 'seated': 1}, {'party': 2, 'timeslot': 1, "
 "'party_size': 8, 'seated': 1}, {'party': 3, 'timeslot': 1, 'party_size': 8, "
 "'seated': 1}, {'party': 4, 'timeslot': 1, 'party_size': 8, 'seated': 1}, "
 "{'party': 5, 'timeslot': 1, 'party_size': 6, 'seated': 0}, {'

In [11]:
# Helper function

def calc_delta_space(removed_size, added_size):
    removed_tables = table_mapping[removed_size]
    added_tables = table_mapping[added_size]
    
    return 3 * (added_tables - removed_tables)

def calc_delta_space2(added_size):
    added_tables = table_mapping[added_size]
    
    return 3 * added_tables


### Parse input parameters for Simulated Annealing

In [12]:
import json

alpha = scale_temp_by = m = k = iterations = 0

with open('../data.json') as f:
    data = json.load(f)
    m = data['m']
    k = data['k']
    alpha = data['alpha']
    scale_temp_by = data['scale_temp_by']
    iterations = m * k

print(f"k = {k}, m = {m}")

k = 6, m = 3


### Simulated annealing pseudocode

for each iteration:
    1. choose a timeslot
    2. pick a candidate table to remove
    3. add tables until we can't anymore
        3.1 find the first party that is not yet seated
        3.2 add that to challenger
        3.3 calc new revenue & space
    4. if revenue inc. or RNG < acceptance prob., then accept candidate solution and append incumbent
        else, don't append incumbent, let it die
    5. make sure challenger removes table to be removed and adds table to added
    6. update iteration no. and temperature (if iteration % 2 == 0)
    
    
    

### Step 7: Run Simulated Annealing

In [13]:
import math
import time

start_time = time.time()

og_revenue = incumbent[8]
T = scale_temp_by*og_revenue


incumbent_history = []
incumbent.append(0)
incumbent_history.append(incumbent)

print(f"{I}")

with open('log' + RG_HEX_ID + '.txt', 'w') as log_file:
    log_file.write(f"Initialized parameters: T_0 = {T:.2f}, alpha = {alpha}, m = {m}, k = {k}, iterations = {m*k}\n\n")
    
    for iteration in range(iterations):
        which_timeslot = random.randint(0,6)
        #incumbent[which_timeslot] = sorted(incumbent[which_timeslot], key=lambda k: k['seated'], reverse=True)
        print(f"old incumbent? {incumbent[which_timeslot]}")
        print(f"TIMESLOT {which_timeslot}")
        log_file.write(f"RANDOMLY GENERATED TIMESLOT: {which_timeslot}\n")
        # pprint.pprint(f"INCUMBENT FOR TIMESLOT {incumbent[which_timeslot]}")
        
        challenger = incumbent.copy()
        
        first_zero = -1
        
        for index, entry in enumerate(challenger[which_timeslot]):
            if entry['seated'] == 0 and index != 0:
                first_zero = index
                break
        if first_zero == -1:
            failures += 1
            # do nothing because all parties are seated
            print("do nothing")
            # while loop could get us into an infinite loop. 
            # This is the easiest way to mitigate since we want a do-while loop which doesn't exist in Python
            if failures == 10:
                break
            continue
        
        challenger = incumbent.copy()

        print(f"first zero: {first_zero}")
        table_to_remove = random.randint(0, first_zero-1)
        removed_party_size = challenger[which_timeslot][table_to_remove]['party_size']
        removed_table_size = table_mapping[challenger[which_timeslot][table_to_remove]['party_size']]
        removed_space = removed_table_size*3 + 6
        
        challenger[which_timeslot][table_to_remove]['seated'] = 10
        
        print(f"evaluating removing table #{table_to_remove}")
        log_file.write(f"evaluating removing table #{table_to_remove}\n")
        
        candidate_change_in_space = removed_space * -1
        candidate_change_in_revenue = removed_party_size * marginal_revenue[removed_party_size] * -1
        # Once we remove a table, keep adding new ones until we don't have any room
        # while loop condition - while we can add at least the smallest table to the restaurant. 
        # Smallest party is composed of 2 people, hence table_mapping[2], where party of 2 is mapped to # tables needed
        num_runs = 0
        
        # no matter what, we can only add 2 additional tables
        while(challenger[9][which_timeslot] + table_mapping[2]*3 + 6 + candidate_change_in_space < MAX_SIZE):
            num_runs += 1
            failures = 0

            first_zero = -1
            for index, entry in enumerate(challenger[which_timeslot]):
                if index != 0:
                    if entry['seated'] == 0:
                        first_zero = index
                        break
            if first_zero == -1:
                failures += 1
                # do nothing because all parties are seated
                print("do nothing")
                # while loop could get us into an infinite loop. 
                # This is the easiest way to mitigate since we want a do-while loop which doesn't exist in Python
                if failures == 10:
                    break
                continue

            print(f"FIRST ZERO {first_zero}")
            
            # this is just the index
            table_to_add = random.randint(first_zero, len(challenger[which_timeslot])-1)
            print(f"evaluating adding table #{table_to_add}")
            log_file.write(f"evaluating adding table #{table_to_add}")
            
            added_party_size = challenger[which_timeslot][table_to_add]['party_size']
            candidate_change_in_space += table_mapping[added_party_size]*3 + 6

            print(f"changing 0 to 1")
            challenger[which_timeslot][table_to_add]['seated'] = 1

            candidate_change_in_revenue += added_party_size * marginal_revenue[added_party_size]
            
#             # Re-sort affected timeslot in descending order 
#             print(f"sorting...\n before\n{challenger[which_timeslot]}\nafter\n{challenger[which_timeslot]}\n\n")
#             challenger[which_timeslot] = sorted(challenger[which_timeslot], key=lambda k: k['seated'], reverse=True)
            
        log_file.write(f"change in space = {candidate_change_in_space}")
        log_file.write(f"change in revenue = {candidate_change_in_revenue}")

        RNG = random.uniform(0, 1)
        
        print(f"\n\n  current = {challenger[which_timeslot]}\n\n challenger = {challenger[which_timeslot]} \n\n")

        if candidate_change_in_revenue > 0:
            print("ACCEPTING CANDIDATE SOLUTION, OBJ VAL INCREASED")
            log_file.write("ACCEPTING CANDIDATE SOLUTION, OBJ VAL INCREASED\n")
            print(f"delta rev {candidate_change_in_revenue}")

            print(f"changing {challenger[which_timeslot][0]['seated']} to 0")
            challenger[which_timeslot][0]['seated'] = 0
            temp = sorted(challenger[which_timeslot], key=lambda k: k['seated'], reverse=True)
            challenger[which_timeslot] = temp
            
            # accept challenger solution - overwrite incumbent
            challenger[8] = revenue + candidate_change_in_revenue
            challenger[9][which_timeslot] = challenger[9][which_timeslot] + candidate_change_in_space
            challenger[-1] = iteration+1
            
            for i in challenger[which_timeslot]:
                if i['seated'] == 10:
                    i['seated'] = 0

            incumbent_history.append(challenger)
            print(f"new incumbent? {challenger[which_timeslot]}")
            
        elif RNG < math.exp(candidate_change_in_revenue / T):
            print(f"change in revenue = {candidate_change_in_revenue}, T = {T}")
            print(f"ACCEPTING CANDIDATE WITH PROBABILITY {RNG} < {math.exp(candidate_change_in_revenue / T)}")
            log_file.write(f"ACCEPTING CANDIDATE WITH PROBABILITY {RNG} < {math.exp(candidate_change_in_revenue / T)}\n")

            print(f"changing {challenger[which_timeslot][0]['seated']} to 0")
            challenger[which_timeslot][0]['seated'] = 0
            temp = sorted(challenger[which_timeslot], key=lambda k: k['seated'], reverse=True)
            challenger[which_timeslot] = temp
            
            # accept challenger solution - add incumbent
            challenger[8] = revenue + candidate_change_in_revenue
            print(f"current spaced occupied for timeslot {which_timeslot} = {challenger[9][which_timeslot]}")
            log_file.write(f"current spaced occupied for timeslot {which_timeslot} = {challenger[9][which_timeslot]}")
            challenger[9][which_timeslot] = challenger[9][which_timeslot] + candidate_change_in_space
            print(f"new spaced occupied for timeslot {which_timeslot} = {challenger[9][which_timeslot]}")
            log_file.write(f"new spaced occupied for timeslot {which_timeslot} = {challenger[9][which_timeslot]}")
            challenger = challenger.copy()
            challenger[-1] = iteration+1
            
            for i in challenger[which_timeslot]:
                if i['seated'] == 10:
                    i['seated'] = 0

            incumbent_history.append(challenger)
            print(f"new incumbent? {incumbent[which_timeslot]}")

        else:
            print("REJECTING CANDIDATE SOLUTION")
            log_file.write("REJECTING CANDIDATE SOLUTION\n")
        print("\n\n")
        


        if (iteration % m == 0):
            T = T*alpha
        print(f"ITERATION = {iteration}")
        print(f"TEMPERATURE = {T}\n\n")
        log_file.write(f"Completed Iteration = {iteration}\n")
        log_file.write(f"Updating T... T_{i} = {T:.2f}\n\n\n")
    
    print("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))
    log_file.write("\n\n--- %s seconds ---\n\n" % (time.time() - start_time))

[10, 12, 14, 14, 15, 15, 12]
old incumbent? [{'party': 0, 'timeslot': 0, 'party_size': 7, 'seated': 1}, {'party': 1, 'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 2, 'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 3, 'timeslot': 0, 'party_size': 5, 'seated': 1}, {'party': 4, 'timeslot': 0, 'party_size': 6, 'seated': 1}, {'party': 5, 'timeslot': 0, 'party_size': 5, 'seated': 1}, {'party': 6, 'timeslot': 0, 'party_size': 4, 'seated': 0}, {'party': 7, 'timeslot': 0, 'party_size': 4, 'seated': 0}, {'party': 8, 'timeslot': 0, 'party_size': 4, 'seated': 0}, {'party': 9, 'timeslot': 0, 'party_size': 2, 'seated': 0}]
TIMESLOT 0
first zero: 6
evaluating removing table #4
FIRST ZERO 6
evaluating adding table #6
changing 0 to 1


  current = [{'party': 0, 'timeslot': 0, 'party_size': 7, 'seated': 1}, {'party': 1, 'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 2, 'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 3, 'timeslot': 0, 'party_size': 5, 'seated': 1}, {'pa

In [14]:
print(f"Total Revenue: ${incumbent[8]}")
print(f"Average Space Used per Timeslot: {sum(incumbent[9]) / 7} m^2")

total_guests = 0
seated_guests = 0
total_groups = 0
seated_groups = 0
party_size_histogram = {
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0
}
for i in range(K):
    for j, val in enumerate(challenger[i]):
        party_size = val['party_size']
        party_size_histogram[party_size] += 1
        total_guests += party_size
        total_groups += 1
        if (val['seated'] == 1):
            seated_guests += party_size
            seated_groups += 1
for key, value in party_size_histogram.items():
    print(f"Party Size: {key}, occurrences: {value}")
    
print(f"Average meal cost: ${challenger[8] / seated_groups}")
print(f"Percentage of groups seated: {seated_groups / total_groups * 100}%")

Total Revenue: $7620
Average Space Used per Timeslot: 100.71428571428571 m^2
Party Size: 2, occurrences: 14
Party Size: 3, occurrences: 14
Party Size: 4, occurrences: 11
Party Size: 5, occurrences: 11
Party Size: 6, occurrences: 13
Party Size: 7, occurrences: 15
Party Size: 8, occurrences: 14
Average meal cost: $173.1818181818182
Percentage of groups seated: 47.82608695652174%


In [15]:
pprint.pprint(incumbent_history)

[[[{'party': 0, 'party_size': 7, 'seated': 10, 'timeslot': 0},
   {'party': 1, 'party_size': 8, 'seated': 0, 'timeslot': 0},
   {'party': 2, 'party_size': 8, 'seated': 1, 'timeslot': 0},
   {'party': 3, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 4, 'party_size': 6, 'seated': 0, 'timeslot': 0},
   {'party': 5, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 6, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 7, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 8, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 9, 'party_size': 2, 'seated': 1, 'timeslot': 0}],
  [{'party': 0, 'party_size': 8, 'seated': 10, 'timeslot': 1},
   {'party': 1, 'party_size': 7, 'seated': 0, 'timeslot': 1},
   {'party': 2, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 3, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 4, 'party_size': 8, 'seated': 1, 'timeslot': 1},
   {'party': 5, 'party_size': 6, 'seated': 0, 'timeslot': 1},
   {'

 [[{'party': 0, 'party_size': 7, 'seated': 10, 'timeslot': 0},
   {'party': 1, 'party_size': 8, 'seated': 0, 'timeslot': 0},
   {'party': 2, 'party_size': 8, 'seated': 1, 'timeslot': 0},
   {'party': 3, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 4, 'party_size': 6, 'seated': 0, 'timeslot': 0},
   {'party': 5, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 6, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 7, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 8, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 9, 'party_size': 2, 'seated': 1, 'timeslot': 0}],
  [{'party': 0, 'party_size': 8, 'seated': 10, 'timeslot': 1},
   {'party': 1, 'party_size': 7, 'seated': 0, 'timeslot': 1},
   {'party': 2, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 3, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 4, 'party_size': 8, 'seated': 1, 'timeslot': 1},
   {'party': 5, 'party_size': 6, 'seated': 0, 'timeslot': 1},
   {'

   {'party': 11, 'party_size': 2, 'seated': 1, 'timeslot': 6}],
  991,
  [105, 102, 102, 102, 96, 99, 99],
  3],
 [[{'party': 0, 'party_size': 7, 'seated': 10, 'timeslot': 0},
   {'party': 1, 'party_size': 8, 'seated': 0, 'timeslot': 0},
   {'party': 2, 'party_size': 8, 'seated': 1, 'timeslot': 0},
   {'party': 3, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 4, 'party_size': 6, 'seated': 0, 'timeslot': 0},
   {'party': 5, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 6, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 7, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 8, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 9, 'party_size': 2, 'seated': 1, 'timeslot': 0}],
  [{'party': 0, 'party_size': 8, 'seated': 10, 'timeslot': 1},
   {'party': 1, 'party_size': 7, 'seated': 0, 'timeslot': 1},
   {'party': 2, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 3, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 4, 

   {'party': 11, 'party_size': 2, 'seated': 1, 'timeslot': 3},
   {'party': 12, 'party_size': 2, 'seated': 0, 'timeslot': 3},
   {'party': 13, 'party_size': 2, 'seated': 0, 'timeslot': 3}],
  [{'party': 0, 'party_size': 8, 'seated': 0, 'timeslot': 4},
   {'party': 1, 'party_size': 8, 'seated': 1, 'timeslot': 4},
   {'party': 2, 'party_size': 7, 'seated': 10, 'timeslot': 4},
   {'party': 3, 'party_size': 7, 'seated': 0, 'timeslot': 4},
   {'party': 4, 'party_size': 8, 'seated': 1, 'timeslot': 4},
   {'party': 5, 'party_size': 7, 'seated': 0, 'timeslot': 4},
   {'party': 6, 'party_size': 7, 'seated': 0, 'timeslot': 4},
   {'party': 7, 'party_size': 6, 'seated': 0, 'timeslot': 4},
   {'party': 8, 'party_size': 5, 'seated': 0, 'timeslot': 4},
   {'party': 9, 'party_size': 5, 'seated': 0, 'timeslot': 4},
   {'party': 10, 'party_size': 3, 'seated': 1, 'timeslot': 4},
   {'party': 11, 'party_size': 4, 'seated': 1, 'timeslot': 4},
   {'party': 12, 'party_size': 3, 'seated': 0, 'timeslot': 4},


 [[{'party': 0, 'party_size': 7, 'seated': 10, 'timeslot': 0},
   {'party': 1, 'party_size': 8, 'seated': 0, 'timeslot': 0},
   {'party': 2, 'party_size': 8, 'seated': 1, 'timeslot': 0},
   {'party': 3, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 4, 'party_size': 6, 'seated': 0, 'timeslot': 0},
   {'party': 5, 'party_size': 5, 'seated': 1, 'timeslot': 0},
   {'party': 6, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 7, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 8, 'party_size': 4, 'seated': 1, 'timeslot': 0},
   {'party': 9, 'party_size': 2, 'seated': 1, 'timeslot': 0}],
  [{'party': 0, 'party_size': 8, 'seated': 10, 'timeslot': 1},
   {'party': 1, 'party_size': 7, 'seated': 0, 'timeslot': 1},
   {'party': 2, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 3, 'party_size': 8, 'seated': 0, 'timeslot': 1},
   {'party': 4, 'party_size': 8, 'seated': 1, 'timeslot': 1},
   {'party': 5, 'party_size': 6, 'seated': 0, 'timeslot': 1},
   {'

In [16]:
max_OV = 0
idx = 0
for index, incumbent in enumerate(incumbent_history):
    if incumbent[8] > max_OV:
        print(f"{incumbent[8]}, max = {max_OV}")
        max_OV = incumbent[8]
        idx = index

print(f"index of max incumbent {max_OV}")
#pprint.pprint(f"{incumbent_history[idx]}")
    

7620, max = 0
index of max incumbent 7620


In [17]:
def write_incumbent_to_file(file_name, index, incumbent):
    with open(file_name, 'a') as csv_file:
        writer = csv.writer(csv_file)
        header = []
        for key in incumbent[0][0].keys():
            header.append(key)
        writer.writerow([f"Incumbent {index}"])
        writer.writerow(header)
        for i in range(7):
            writer.writerow([f"Timeslot {i+1}"])
            for j in incumbent[i]:
                li = list(j.values())
                writer.writerow(li)
                #for val in j.values():
                #writer.writerow(j)
        writer.writerow([f"Objective Function value: {incumbent[8]}"])
        for i in range(3):
            writer.writerow('')

file_name = test = "simulated_annealing_" + RG_HEX_ID + ".csv"
for idx, inc in enumerate(incumbent_history):
    write_incumbent_to_file(file_name, idx, inc)
with open(file_name, 'a') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["party i", "size of Party i", "timeslot k", "seated (binary)", "added revenue", "cum. revenue"])

In [18]:
print(incumbent[which_timeslot])

[{'party': 0, 'timeslot': 0, 'party_size': 7, 'seated': 10}, {'party': 1, 'timeslot': 0, 'party_size': 8, 'seated': 0}, {'party': 2, 'timeslot': 0, 'party_size': 8, 'seated': 1}, {'party': 3, 'timeslot': 0, 'party_size': 5, 'seated': 1}, {'party': 4, 'timeslot': 0, 'party_size': 6, 'seated': 0}, {'party': 5, 'timeslot': 0, 'party_size': 5, 'seated': 1}, {'party': 6, 'timeslot': 0, 'party_size': 4, 'seated': 1}, {'party': 7, 'timeslot': 0, 'party_size': 4, 'seated': 1}, {'party': 8, 'timeslot': 0, 'party_size': 4, 'seated': 1}, {'party': 9, 'timeslot': 0, 'party_size': 2, 'seated': 1}]
